In [ ]:
import time
import requests
import json

import os
from multiprocessing import Process, current_process
import multiprocessing as mp

In [ ]:
from urllib.request import urlopen
pic_url = res.url

img = urlopen(pic_url)

In [ ]:
res.url

In [ ]:
res = requests.get('https://api.unsplash.com/search/photos', data={'query': 'pizza'})
res.text

In [ ]:
user_ids = ['95976927',
 '21437831',
 '420432840',
 '219695265',
 '812607',
 '124355785',
 '874794',
 '2736290',
 '33223831',
 '72092127',
 '137688760',
 '110387926',
 '9080587',
 '10373310',
 '156529191',
 '9802117',
 '115064549',
 '33941526',
 '71346506',
 '444993253',
 '46906362',
 '27177776',
 '23181729',
 '172512309',
 '23486688',
 '169344787',
 '121972290',
 '61740821',
 '57131939',
 '317481495',
 '76776815',
 '109282801',
 '55505578',
 '81515937',
 '93569231',
 '21022885',
 '555173130',
 '4726755',
 '353517397',
 '295754416',
 '2867115',
 '42271250',
 '12737975',
 '108518309',
 '68035479',
 '513533165',
 '362769',
 '293688379',
 '114169708',
 '367416238']

In [ ]:
import requests
from io import BytesIO
from PIL import Image
import pytesseract
from Levenshtein import distance
import numpy as np
from time import sleep


text_height_threshold = 0.032


def vk_api(method, data):
    access_token = 'f54fe3b72906d9d9194ad529c1cd65d8f20e0bee7086f93a8d01ebea4ca66940af1eca4bcf21887f02a66'
    version = '5.37'

    link = "https://api.vk.com/method/{}?access_token={}&v={}".format(method, access_token, version)
    for key, val in data.items():
        link += '&{}={}'.format(key, str(val))

    res = requests.get(link)
    return res.json()


# находим id всех пользователей

def get_all_members(group_id):
    group_id = str(group_id)
    if '/' in group_id:
        group_id = group_id.split('/')[-1]

    members = []
    old_members = [0] * 1000
    i = 0

    while len(old_members) == 1000:
        data = {'group_id': group_id,
                'offset': i * 1000,
                'count': 1000}

        try:
            old_members = vk_api('groups.getMembers', data)
            old_members = old_members['response']['items']
        except Exception as e:
            if old_members['error']['error_code'] == 15:
                break
            else:
                print('error: old_members = ', old_members)
                raise ValueError(str(e))
        i += 1
        members += old_members
        sleep(0.5)

    return members


# находим все сертификаты одного пользователя

def download_image(image_url):
    
    res = requests.get(image_url)
    image = Image.open(BytesIO(res.content))
    
    return image


def images_from_res(res):
    images_paths = []
    for index, item in enumerate(res):
        try:
            attachments = item['attachments']

            for attachment in attachments:
                versions_of_photo = []
                if attachment['type'] == 'photo':
                    photo = attachment['photo']
                    for key, path in photo.items():
                        if 'photo' in key:
                            versions_of_photo.append({'size': int(key.replace('photo_', '')), 'path': path})

                # находим изображение с самым большим разрешением
                versions_of_photo = sorted(versions_of_photo, key=lambda k: k['size'], reverse=True)
                best_photo_path = versions_of_photo[0]['path']
                smallest_photo_path = versions_of_photo[-1]['path']

                # post_url = 'https://vk.com/id{}?w=wall{}_{}'.format(item['from_id'], item['from_id'], item['id'])
                images_paths.append({'post_id': item['id'],
                                     'user_id': item['from_id'],
                                     'image_url': best_photo_path,
                                     'preview_url': smallest_photo_path})
        except Exception as e:
            pass

    return images_paths


def find_in_text(text, target_words=['сертификат']):
    for target_word in target_words:
        for line in text.split('\n'):
            for word in line.split(' '):
                if len(word) > 5:
                    word = word.lower()
                    if target_word in word:
                        return True, target_word

                    dist = distance(word, target_word)
                    if dist < 2:
                        return True, target_word
    return False, ''


def get_cert_bbs(image):
    data = pytesseract.image_to_data(image, lang='rus').split('\n')
    all_rows = [row.split('\t') for row in data]

    column_names = all_rows[0]
    data_rows = all_rows[1:]

    texts_data = [dict(zip(column_names, row)) for row in data_rows]

    bbs = []
    max_text_height = 0
    for text_data in texts_data:
        try:
            if len(text_data['text'].replace(' ', '')) > 2:
                img_w = image.size[0]
                img_h = image.size[1]
                bbs.append({'text': text_data['text'],
                            'y': int(text_data['top']) / img_h,
                            'x': int(text_data['left']) / img_w,
                            'w': int(text_data['width']) / img_w,
                            'h': int(text_data['height']) / img_h})
                if (int(text_data['height']) / img_h) > max_text_height:
                    max_text_height = int(text_data['height']) / img_h
        except Exception as e:
            pass

    return bbs, max_text_height


def analyze_user(user_id, count=30, offset=0, target_words=['сертификат']):
    certificates_data = []

    data = {'filter': 'owner',
            'extended': '0',
            'owner_id': user_id,
            'count': count,
            'offset': offset}
    res_posts = vk_api('wall.get', data)

    try:
        images_data = images_from_res(res_posts['response']['items'])

        for image_data in images_data:
            global images_cnt
            images_cnt += 1
            image_url = image_data['image_url']

            start = time.time()
            image = download_image(image_url)
            end = time.time()
            global download_time
            download_time += end - start
            #print('скачивание: ', end-start)

            start = time.time()
            image_for_detect = deepcopy(image)
            #image_for_detect.thumbnail((512, 512), Image.ANTIALIAS)
            image_for_detect = image_for_detect.crop((0, 0, image_for_detect.size[0], image_for_detect.size[1]/2))
            text_from_img = pytesseract.image_to_string(image_for_detect, lang='rus+eng')
            end = time.time()
            global analyze_time
            analyze_time += end - start
            #print('анализ: ', end-start)

            is_finded, trigger_word = find_in_text(text_from_img, target_words=target_words)
            if is_finded:
                bbs, max_text_height = get_cert_bbs(image)

                if max_text_height < text_height_threshold:
                    is_hidden = True
                else:
                    is_hidden = False

                image_data.update({'text_from_image': text_from_img, 'bbs': bbs, 'trigger_word': trigger_word, 'is_hidden': is_hidden})

                certificates_data.append(image_data)

    except Exception as e:

        try:
            error_msg = res_posts['error']['error_msg']
            error_code = res_posts['error']['error_code']

        except Exception:
            error_msg = str(e)
            error_code = None

        print('error! user_id: {} err_code: {} msg: {}'.format(user_id, error_code, error_msg))
        if error_code == 29:  # если достигли лимита запросов в день
            raise ValueError(error_msg)

        return []

    return certificates_data


# кластеризуем все изображения и запишем результат в БД

def diff_iou(bb1, bb2):
    """
    bb : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    """

    if (bb1 == None) or (bb2 == None):
        return 0.0
    # determine the coordinates of the intersection rectangle
    bb1 = {'x1': bb1['x'], 'x2': bb1['x'] + bb1['w'], 'y1': bb1['y'], 'y2': bb1['y'] + bb1['h'], 'text': bb1['text']}
    bb2 = {'x1': bb2['x'], 'x2': bb2['x'] + bb2['w'], 'y1': bb2['y'], 'y2': bb2['y'] + bb2['h'], 'text': bb2['text']}

    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both BBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    l1 = len(bb1['text'])
    l2 = len(bb2['text'])
    levin_dist = 1 - distance(bb1['text'], bb2['text']) / (l1 + l2)
    iou = iou * levin_dist

    assert iou >= 0.0
    assert iou <= 1.0
    return iou


def find_longer_dist(matrix):
    """
        функция для нахождения пути (в матрице) при котором сумма всех нод данного пути будет максимальна
    """

    # идем горизонтальными полосами
    non_zero_indexes = matrix[0].nonzero()[0]

    if matrix.shape[0] > 1:
        if len(non_zero_indexes) == 0:
            return find_longer_dist(matrix[1:])
        else:
            lens = []
            for index in non_zero_indexes:
                len_through_index = find_longer_dist(matrix[1:])
                lens.append(len_through_index)
            return max(matrix[0]) + max(lens)

    else:
        if len(non_zero_indexes) == 0:
            # если в последнем слое только нули - возвращаем текущую длинну пути
            return 0
        else:
            # ищем максимальную длинну в последнем слое
            lens = []
            for index in non_zero_indexes:
                len_through_index = matrix[0][index]
                lens.append(len_through_index)
            return max(lens)


def diff_bbs(bbs1, bbs2):
    """
        0 - похожи, 1 - не похожи
    """
    matrix = np.zeros((len(bbs1), len(bbs2)))

    for i, bb1 in enumerate(bbs1):
        for j, bb2 in enumerate(bbs2):
            matrix[i][j] = diff_iou(bb1, bb2)

    similarity_rows = 1 - find_longer_dist(matrix) / matrix.shape[0]
    similarity_columnes = 1 - find_longer_dist(matrix) / matrix.shape[0]
    similarity = max(similarity_rows, similarity_columnes)
    return similarity

In [ ]:
%%time
download_time = 0
analyze_time = 0
images_cnt = 0

def download_image(image_url):
    res = requests.get(image_url)
    image = Image.open(BytesIO(res.content))

    return image

res = analyze_user(user_ids[4], target_words=['диплом',
                                        'сертификат',
                                        'лицензия',
                                        'certified',
                                        'specialist',
                                        'специалист',
                                        'эксперт'])

print('download_time = {}\nanalyze_time = {}\nimages_cnt = {}\ncerts_cnt = {}'.format(download_time, analyze_time, images_cnt, len(res)))

In [ ]:
im = download_image(res[0]['image_url'])
im = im.crop((0, 0, im.size[0], im.size[1]/2))
im.show()

In [ ]:
print('download_time = {}\nanalyze_time = {}\nimages_cnt = {}\ncerts_cnt = {}'.format(download_time, analyze_time, images_cnt, len(res)))
# 26 - 512 + crop_0.5h
# 26 - 512
#  - realsize + crop_0.5h
# 56 - realsize

In [ ]:
%%time

treshold_delta = 0.5
download_time = 0
def mp_analyze_user(user, args, return_dict):
    global last_moment
    time_delta = time.time() - last_moment[user['id']]
    last_moment = time.time()
    if time_delta < treshold_delta:
        time.sleep(treshold_delta)

    user_id = args
    print('user_id={}'.format(user_id))

    certs_of_user = analyze_user(user_id, target_words=['диплом',
                                                        'сертификат',
                                                        'лицензия',
                                                        'certified',
                                                        'specialist',
                                                        'специалист',
                                                        'эксперт'])
    '''headers = {'Content-type': 'application/json',
               'Accept': 'text/plain',
               'Content-Encoding': 'utf-8'}
               
    certs_of_user = requests.post('http://0.0.0.0:5050/analyze', data=json.dumps({'user_id': user_id}), headers=headers).json()['response']'''
                                                        

    for cert_data in certs_of_user:
        try:
            cluster_id = 1 #clusterize(cert_data)
            cert_data.update({'cluster_id': cluster_id})
            #add_certificate(cert_data)

        except Exception as e:
            # если новых данных нет
            print(str(e))

    global start_time
    print((int(time.time())-start_time), ' => complete ', user_id, 'finded certs_cnt = ', len(certs_of_user))


def analyze_users(users_ids, proc_cnt):
    users = get_users_fn() # requests.get('http://78.47.176.17:14288/api/get_all_users').json()

    procs = []
    manager = mp.Manager()
    return_dict = manager.dict()
    users_cnt = len(users_ids)
    while len(users_ids) > 0:
        #update_status(session_id, '{}%'.format(int(1000*(users_cnt-len(users_ids))/users_cnt)/10))
        print('current user ind: {} from {}'.format(users_cnt-len(users_ids), users_cnt))

        for user in users:
            if len(users_ids) == 0:
                break
            arg = users_ids.pop()
            proc = Process(target=mp_analyze_user, args=(user, arg, return_dict,))
            procs.append(proc)
            proc.start()
        
        for proc in procs:
            proc.join()


def get_users_fn():
    return base_users[:10] # requests.get('http://78.47.176.17:14288/api/get_all_users').json()

start_time = int(time.time())
last_moment = {user['id']: time.time() for user in get_users_fn()}
analyze_users(user_ids[:50], 10)
#for 4 users
#2m9c - multi (5)
#3m40c - single
#4m7c - old method

#for 50 users
#17m55c - multi (5)
#16m20c - multi(10)
#43m30c - single
#34m50c - old method

# 9m30c исправлено время ожидания - для каждого user оно отдельное. анализ делается через api
# 9m27c исправлено время ожидания - для каждого user оно отдельное. использовано 5 токенов
# 8m48c исправлено время ожидания - для каждого user оно отдельное. использовано 10 токенов
# 6m53c исправлено время ожидания - для каждого user оно отдельное. использовано 10 токенов, сжатие до 512х512

In [ ]:
base_users = []
for i, token in enumerate(tokens.split(' ')[1:]):
    base_users.append({'id': i, 'query': None, 'query_class': None, 'token': token, 'vk_id': i})

In [ ]:
import time
import requests
import json

import os
from multiprocessing import Process, current_process
import multiprocessing as mp

user_ids = [file_name.split('.')[0] for file_name in os.listdir('saved_data')]

In [ ]:
%%time

def vk_api(method, data, access_token='f54fe3b72906d9d9194ad529c1cd65d8f20e0bee7086f93a8d01ebea4ca66940af1eca4bcf21887f02a66', old_time=time.time()):

    version = '5.37'

    link = "https://api.vk.com/method/{}?access_token={}&v={}".format(method, access_token, version)
    for key, val in data.items():
        link += '&{}={}'.format(key, str(val))

    res = requests.get(link)

    time_delta = time.time() - old_time
    if time_delta < 0.333:
        time.sleep(0.333 - time_delta)
    return res.json(), time.time()

data = {'group_id': 'sci',
        'offset': 0,
        'count': 1000}

old_time = time.time()
for i in range(20):
    try:
        res, old_time = vk_api('groups.getMembers', data, old_time=old_time)
        res = res['response']['items']
    except Exception as e:
        print(str(e))
        print(i)
        print(res)
        break

In [ ]:
users = requests.get('http://78.47.176.17:14288/api/get_all_users').json()

for user in users:
    user['old_time'] = time.time()

users

In [ ]:
vk_api('groups.getById', {'group_id': '163528123', 'fields': 'members_count'})[0]['response'][0]['members_count']

In [ ]:
%%time

 
def get_users(token, user):
    data = {'group_id': 'sci',
            'offset': 0,
            'count': 1000}

    try:
        old_time = user['old_time']
        res, old_time = vk_api('groups.getMembers', data, token, old_time)
        user['old_time'] = old_time

        with open('multiproc.txt') as f:
            old_data = f.read()
        new_data = old_data + '\n' + str(old_time)
        with open('multiproc.txt', 'w') as f:
            f.write(new_data)

        res = res['response']['items']
        res = 'success'
    except Exception as e:
        pass
    proc_name = current_process().name
    print('res: {}\nproc_name: {}\n'.format(res, proc_name))
 

if __name__ == '__main__':
    for user in users:
        user['old_time'] = time.time()

    procs = []
    
    for i in range(20 // len(users)):
        for user in users:
            token = user['token']
            proc = Process(target=get_users, args=(token, user,))
            procs.append(proc)
            proc.start()
        
        for proc in procs:
            proc.join()

In [ ]:
%%time


class ParallelWorker:
    def __init__(self):
        self.break_var = False

    def get_users(self, user, args, return_dict):
        data = {'group_id': args['group_id'],
                'count': 1000,
                'offset': args['offset']}
        
        try:
            old_time = user['old_time']
            token = user['token']
            res, old_time = vk_api('groups.getMembers', data, token, old_time)
            user['old_time'] = old_time
            
            res = res['response']['items']


            proc_name = current_process().name
            with open('multiproc.txt') as f:
                
                old_data = f.read()
            new_data = old_data + '\n' + str(proc_name) + ' => ' + str(res)
            with open('multiproc.txt', 'w') as f:
                f.write(new_data)

            
            return_dict[proc_name] = res
        except Exception as e:
            print('error!\n', str(e))
            pass
        
    
    
    def parallel_def(self, fn, args=[]):
        users = requests.get('http://78.47.176.17:14288/api/get_all_users').json()

        for user in users: # брать и записывать в бд
            user['old_time'] = time.time()

        procs = []
        manager = mp.Manager()
        return_dict = manager.dict()
        self.break_var = False
        while len(args['var']) > 0:
            for user in users:
                if len(args) == 0:
                    break
                arg = args['var'].pop()
                arg.update(args['static'])
                proc = Process(target=fn, args=(user, arg, return_dict,))
                procs.append(proc)
                proc.start()
            
            for proc in procs:
                proc.join()

        return [val for arr in return_dict.values() for val in arr]

parallel_worker = ParallelWorker()
getted_users = parallel_worker.parallel_def(parallel_worker.get_users, args={'var': [{'offset': i} for i in range(0, 10000, 1000)], 'static': {'group_id': '163528123'}})

In [ ]:
time.time()

In [ ]:
%%time
    
def mp_analyze_user(user, args, return_dict):
    global last_moment
    time_delta = time.time() - last_moment
    last_moment = time.time()
    if time_delta < 0.3:
        time.sleep(0.3)

    user_id = args
    print('user_id={}'.format(user_id))

    certs_of_user = analyze_user(user_id, target_words=['диплом',
                                                        'сертификат',
                                                        'лицензия',
                                                        'certified',
                                                        'specialist',
                                                        'специалист',
                                                        'эксперт'])
                                                        

    for cert_data in certs_of_user:
        try:
            cluster_id = 1 #clusterize(cert_data)
            cert_data.update({'cluster_id': cluster_id})
            #add_certificate(cert_data)

        except Exception as e:
            # если новых данных нет
            print(str(e))

    global start_time
    print((int(time.time())-start_time), ' => complete ', user_id, 'finded certs_cnt = ', len(certs_of_user))


def analyze_users(users_ids, proc_cnt):

    procs = []
    manager = mp.Manager()
    return_dict = manager.dict()
    users_cnt = len(users_ids)
    while len(users_ids) > 0:
        #update_status(session_id, '{}%'.format(int(1000*(users_cnt-len(users_ids))/users_cnt)/10))
        print('current user ind: {} from {}'.format(users_cnt-len(users_ids), users_cnt))

        for i in range(proc_cnt):
            if len(users_ids) == 0:
                break
            arg = users_ids.pop()
            proc = Process(target=mp_analyze_user, args=(user, arg, return_dict,))
            procs.append(proc)
            proc.start()
        
        for proc in procs:
            proc.join()


def get_users_fn():
    return requests.get('http://78.47.176.17:14288/api/get_all_users').json()

start_time = int(time.time())
last_moment = time.time()
analyze_users(user_ids[:50], 10)
#for 4 users
#2m9c - multi (5)
#3m40c - single
#4m7c - old method

#for 50 users
#17m55c - multi (5)
#16m20c - multi(10)
#43m30c - single
#34m50c - old method

In [ ]:
%%time

def get_users_fn():
    return [requests.get('http://78.47.176.17:14288/api/get_all_users').json()[0]]
    
start_time = int(time.time())
analyze_users(user_ids[:50])

In [ ]:
%%time
for user_id in user_ids[:50]:
    certs_of_user = analyze_user(user_id, target_words=['диплом',
                                                        'сертификат',
                                                        'лицензия',
                                                        'certified',
                                                        'specialist',
                                                        'специалист',
                                                        'эксперт'])
                                                        

    for cert_data in certs_of_user:
        try:
            cluster_id = 1 #clusterize(cert_data)
            cert_data.update({'cluster_id': cluster_id})
            #add_certificate(cert_data)

        except Exception as e:
            # если новых данных нет
            print(str(e))

In [ ]:
user_ids = [file_name.split('.')[0] for file_name in os.listdir('saved_data')]
len(user_ids)

In [ ]:
[i for i in range(0, 80, 1000)]

In [ ]:
len(list(set(getted_users)))

In [ ]:
a = [1,2,3,4,5]
a[2:4]

In [ ]:
len(list(set(getted_users)))

In [ ]:
(*[i for i in range(5)])

In [ ]:
items = Session.query.filter(not_(Session.status.contains('in_queue'))).all()
json.loads(json.dumps(items, cls=AlchemyEncoder))


In [ ]:
items = Session.query.filter(and_(not_(Session.status.contains('in_queue')), not_(Session.status.contains('complete')))).first()
json.loads(json.dumps(items, cls=AlchemyEncoder))


In [ ]:
session = Session(status='success', data=['1111', '2222'])

try:
    db.session.add(session)
    db.session.commit()
    db.session.rollback()
except Exception as e:
    print(str(e))

In [ ]:
from sqlalchemy import or_, not_, and_

#filter(or_(*[Сertificate.id.like(cert_id) for cert_id in filtred_cert_ids]))